In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# install a missing dependency
!pip install omegaconf
# restart notebook on first run


Defaulting to user installation because normal site-packages is not writeable


In [3]:
# hack the paths, no package install needed
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [4]:
import logging
logging.basicConfig(level=logging.INFO)
import os

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

/tmp/ipykernel_20946/3545559080.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Authenticate

In [13]:
# type in your COSMOS credentials
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO:astroquery:OK
INFO:astroquery:Login to Euclid data service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]


INFO:astroquery:OK
INFO:astroquery:Login to Euclid cutout service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]


INFO:astroquery:OK


INFO: OK [astroquery.utils.tap.core]


### Set Pipeline Options and Load External Targets

In [14]:
# you can use this config and external target list for a minimal test on six targets within one tile...

# cfg_dict = dict(
#     # TODO change this path to wherever you want to save everything
#     base_dir='/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version',
#     name='external_targets_debug',
#     release_name='F-006', 
#     bands=['VIS', 'NIR_Y'],
#     auxillary_products=['MERPSF','MERRMS','MERBKG'],
    
#     # these will be filled in automatically using on base_dir
#     download_dir=None,
#     tile_dir=None,
#     cutout_dir=None,
#     fits_dir=None,
#     sanity_dir=None
    
# )
# cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
# pipeline.create_folders(cfg)  # fill in those missing paths

# external_targets = pd.DataFrame([
# {
#   'id_str': 'a',
#   'target_ra': 92.8371844681179,
#   'target_dec': -47.8868920683699,
#   'target_field_of_view': 20},
#  {,
#   'id_str': 'b',
#   'target_ra': 92.836691187603,
#   'target_dec': -48.1117628801209,
#   'target_field_of_view': 20},
#  {
#   'id_str': 'd',
#   'target_ra': 92.835560566775,
#   'target_dec': -47.978437467137,
#   'target_field_of_view': 20},
#  {
#   'id_str': 'e',
#   'target_ra': 92.8351734327103,
#   'target_dec': -48.1054532354171,
#   'target_field_of_view': 20},
#  {
#   'id_str': 'f',
#   'target_ra': 92.8343976295701,
#   'target_dec': -47.9993017111107,
#   'target_field_of_view': 20}
# ])

In [15]:
# or you can use this config to run for real

cfg_dict = dict(
    base_dir='/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline',
    name='latest',
    release_name=None,  # allow tiles from any release
    bands=['VIS', 'NIR_Y'],  
    auxillary_products=['MERPSF','MERRMS','MERBKG'],
    
    # ignore these
    download_dir=None,
    tile_dir=None,
    cutout_dir=None,
    fits_dir=None,
    sanity_dir=None
    
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
pipeline.create_folders(cfg)  # fill in those missing paths

# TODO use your own csv, it just needs the columns specified here
external_targets = pd.read_csv('/media/home/my_workspace/combined_karina_strong_lens_classifications.csv', usecols=['id_str', 'target_ra', 'target_dec'])
external_targets['target_field_of_view'] = 20  # arcseconds

INFO:root:Saving to /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest


In [16]:
len(external_targets)

6541

### For each target, find a tile that covers it

In [17]:
targets_with_tiles = pipeline.get_matching_tiles(cfg, external_targets)
targets_with_tiles

INFO:astroquery:Query finished.
INFO:root:Begin target/tile cross-match


INFO: Query finished. [astroquery.utils.tap.core]
Found 2514  query results


INFO:root:Matched 6541 targets to 87 tiles
INFO:root:Targets with tile matches: 6541


,id_str,target_ra,target_dec,target_field_of_view,tile_index
0,EUC_0006_DESI-sel,149.829040,2.154270,20,101544256
1,EUC_0193_SDSS,150.013360,2.029592,20,101542817
2,EUC_0200_SDSS,150.250920,1.864015,20,101541378
3,EUC_0250_DESI-sel,149.945542,2.017199,20,101542817
4,EUC_0342_DESI-sel,150.438411,2.643070,20,101547137
...,...,...,...,...,...
6536,EUC_6111_DESI-sel,150.581806,2.656827,20,101547138
6537,EUC_6112_DESI-sel,150.582986,2.631492,20,101545699
6538,EUC_6119_DESI-sel,150.601048,2.664884,20,101547138
6539,EUC_6121_DESI-sel,150.607258,2.674481,20,101547138


The speed of the pipeline depends on the number of tiles to download. It takes a few minutes per tile.

In [18]:
targets_with_tiles['tile_index'].nunique()

87

### For every tile, download the flux and auxillary (PSF, RMS, BKG) data and make cutouts around each target in that tile

In [ ]:
pipeline.make_cutouts(cfg, targets_with_tiles)

INFO:root:Tile 101544256, 0 of 87
INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
Found 2  query results
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101544256-8DD33D_20240203T001101.427300Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
200 
Reading...


INFO:root:EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101544256-8DD33D_20240203T001101.427300Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101544256-8DD33D_20240203T001101.427300Z_00.00.fits


Done.
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_BGSUB-MOSAIC-VIS_TILE101544256-F5FCD6_20240203T024726.998416Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
200 
Reading...


INFO:root:EUC_MER_BGSUB-MOSAIC-VIS_TILE101544256-F5FCD6_20240203T024726.998416Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-VIS_TILE101544256-F5FCD6_20240203T024726.998416Z_00.00.fits


...

Done.
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_CATALOG-PSF-NIR-Y_TILE101547137-C85368_20240202T132426.399887Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
200 
Reading...


INFO:root:EUC_MER_CATALOG-PSF-NIR-Y_TILE101547137-C85368_20240202T132426.399887Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-NIR-Y_TILE101547137-C85368_20240202T132426.399887Z_00.00.fits


Done.
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_MOSAIC-NIR-Y-RMS_TILE101547137-888B8B_20240202T040809.531263Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
200 
Reading...


INFO:root:EUC_MER_MOSAIC-NIR-Y-RMS_TILE101547137-888B8B_20240202T040809.531263Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-NIR-Y-RMS_TILE101547137-888B8B_20240202T040809.531263Z_00.00.fits


Done.
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_BGMOD-VIS_TILE101547137-E5BA44_20240202T122759.574870Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
